In [1]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
import zipfile
device='cpu'

In [2]:
def load_data_jay_lyrics(): 
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zif:
        with zif.open('jaychou_lyrics.txt') as f:
            corpus_chars =f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    
    return corpus_indices,char_to_idx,idx_to_char,vocab_size

In [3]:
corpus_indices, char_to_idx, idx_to_char, vocab_size=load_data_jay_lyrics()

In [4]:
def one_hot(x,n_class,dtype=torch.float32):
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

In [5]:
def to_onehot(x,n_class):
    return [one_hot(x[:,i],n_class) for i in range(x.shape[1])]

In [6]:
x=torch.arange(10).view(2,5)
to_onehot(x,10)

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])]

In [7]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size

In [8]:
def get_params():
    def _one(shape):
        ts=torch.tensor(np.random.normal(0,0.01,size=shape),dtype=torch.float32)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    W_xh=_one((num_inputs,num_hiddens))
    W_hh=_one((num_hiddens,num_hiddens))
    b_h=torch.nn.Parameter(torch.zeros(num_hiddens,requires_grad=True))
    W_hq=_one((num_hiddens,num_outputs))
    b_q=torch.nn.Parameter(torch.zeros(num_outputs,requires_grad=True))
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])

In [9]:
def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [10]:
def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q=params
    H,=state
    outputs=[]
    for x in inputs:
        H=torch.tanh(torch.matmul(x,W_xh)+torch.matmul(H,W_hh)+b_h)
        Y=torch.matmul(H,W_hq)+b_q
        outputs.append(Y)
    return outputs,(H,)

In [11]:
state=init_rnn_state(x.shape[0],num_hiddens,device)
inputs=to_onehot(x,vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
print(len(outputs),outputs[-1].shape,state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


In [12]:
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx):
    state=init_rnn_state(1,num_hiddens,device)
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X=to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y,state)=rnn(X,state,params)
        if t < len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [13]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开宠禁托吸千凯哭刀真寒'

In [14]:
def grad_clipping(params,theta,device):
    norm=torch.Tensor([0.0],device=device)
    for p in params:
        norm +=(p.grad.data **2).sum()
    norm=norm.sqrt().item()
    if norm > theta:
        for p in params:
            p.grad.data *= (theta/norm)

In [15]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.Tensor(corpus_indices,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [16]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad / batch_size

In [ ]:
def train_and_predict_rnn(rnn, get_params, init_run_state, num_hiddens,
                          vocab_size, corpus_indices, idx_to_char, 
                          char_to_idx,
                          is_random_iter, num_epochs,num_steps, lr, clipping_theta,
                          batch_size, pred_period, pred_len, prefixes):
    if is_random_iter:
        data_iter_fn=data_iter_random
    else:
        data_iter_fn=data_iter_consecutive
    
    params=get_params()
    loss=nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
        for X,Y in data_iter:
            if is_random_iter:
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            inputs=to_onehot(X,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            sgd(params,lr,1)
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
            
        if (epoch +1) % pred_period ==0:
            print('epoch %d ,perplexity %f ,time %.2f sec ' % (epoch +1,math.exp(l_sum/n),time.time()-start))
            
            for prefix in prefixes:
                print('-',predict_rnn(prefix,pred_len,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx))
        

In [17]:
num_epochs,num_steps,batch_size,lr,clipping_theta=250,35,32,1e2,1e-2
pred_period,pred_len,prefixes=50,50,['分开','不分开']

In [ ]:
train_and_predict_rnn(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,
                     corpus_indices,idx_to_char,char_to_idx,None,
                     num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,
                     pred_len,prefixes)

In [24]:
def train_and_predict_rnn2(rnn, get_params, init_run_state, num_hiddens,
                          vocab_size, corpus_indices, idx_to_char, 
                          char_to_idx,
                          is_random_iter, num_epochs,num_steps, lr, clipping_theta,
                          batch_size, pred_period, pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = data_iter_random
    else:
        data_iter_fn = data_iter_consecutive
    
    params=get_params()
    loss=nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        if not is_random_iter :
            state=init_rnn_state(batch_size,num_hiddens,device)
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_fn(corpus_indices,batch_size,num_steps)
        for x,y in data_iter:
            if is_random_iter :
                state=init_rnn_state(batch_size,num_hiddens,device)
            else:
                for s in state:
                    s.detach_()
            inputs=to_onehot(x,vocab_size)
            (outputs,state)=rnn(inputs,state,params)
            outputs=torch.cat(outputs,dim=0)
            print('y shape : ',y.shape)
            print('y : ',y)
            y=torch.transpose(y,0,1).contiguous().view(-1)
            l=loss(outputs,y.long())
            
            if params[0].grad is not None:
                for p in params:
                    p.grad.data.zero_()
            l.backward()
            grad_clipping(params,clipping_theta,device)
            sgd(params,lr,1)
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
            
        if (epoch+1) % pred_period ==0:
            print('epoch %d ,perplexity  %f ,time %.2f sec' % (epoch+1,math.exp(l_sum/n),time.time()-start))
            
            for prefix in prefixes:
                print('- ',predict_rnn(prefix, pred_len, rnn, params, init_run_state,
                                num_hiddens, vocab_size, device ,idx_to_char,
                                char_to_idx))
        

In [25]:
num_epochs,num_steps,batch_size,lr,clipping_theta=250,35,32,1e2,1e-2
pred_period,pred_len,prefixes=50,50,['分开','不分开']

In [26]:
train_and_predict_rnn2(rnn,get_params,init_rnn_state,num_hiddens,vocab_size,
                     corpus_indices,idx_to_char,char_to_idx,None,
                     num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,
                     pred_len,prefixes)

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

...

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5

y shape :  torch.Size([32, 35])
y :  tensor([[179., 926., 971.,  ...,  69., 663., 284.],
        [395.,  87., 992.,  ...,  87., 131., 430.],
        [873., 678., 859.,  ...,  87., 616.,  91.],
        ...,
        [177.,  69., 267.,  ..., 994., 707., 646.],
        [963., 936., 779.,  ..., 463., 983., 702.],
        [333.,  69., 188.,  ...,  69., 554., 815.]])
y shape :  torch.Size([32, 35])
y :  tensor([[142., 284., 142.,  ..., 702., 691., 446.],
        [273., 629.,  69.,  ..., 992., 663.,  70.],
        [ 69.,  51., 321.,  ..., 341.,  69., 158.],
        ...,
        [499., 407., 133.,  ...,  87., 740., 883.],
        [130., 955.,  69.,  ..., 167., 196.,  69.],
        [720., 875., 963.,  ..., 647.,  69., 138.]])
y shape :  torch.Size([32, 35])
y :  tensor([[ 69., 992., 663.,  ...,  69., 320., 395.],
        [567.,  87., 131.,  ...,  69., 746., 159.],
        [915., 702.,  35.,  ..., 104., 927., 514.],
        ...,
        [ 83.,  69., 239.,  ..., 423., 609., 391.],
        [ 29., 5